- pd.merge()、df.join()、pd.concat()、df.append()
- df.reset_index()

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#英文-中文球队名字典
def get_namedict():
    name_dict = {}
    with open('../数据收集/english_name-to-chinese_name.txt','r',encoding='utf-8') as f:
        for line in f.read().split('\n'):
            if line:
                e,c = line.split('-')
                name_dict[e]=c
    return name_dict

def merge_one_df(df):
    df_group = df.groupby('GAME_ID')
    df_result=pd.DataFrame()
#     columns = df.columns.append(df.columns.map(lambda x: 'Away_'+x))
    for game_id in df['GAME_ID'].unique():
        df_x_game = df_group.get_group(game_id)
        df_host = df_x_game[df_x_game['MATCHUP'].str.contains('vs.')]
        df_away = df_x_game[df_x_game['MATCHUP'].str.contains('@')]
        df_result = df_result.append(pd.merge(df_host,df_away,on=['GAME_ID','GAME_DATE','SEASON_ID','MIN','VIDEO_AVAILABLE'],suffixes=['','_Away']),ignore_index=True)
#     df_result.columns = columns
    return df_result

In [3]:
year_dirs = ['../数据收集/'+f for f in os.listdir('../数据收集') if '.' not in f]
name_dict = get_namedict()
for year in year_dirs:
    df_nba = pd.read_excel(year+'/games.xlsx')
    df_lottery = pd.read_excel(year+'/lottery_results.xlsx')
    
    #添加中文球队名称，然后同一场比赛合并为一行
    df_nba['TEAM_NAME_中文'] = df_nba['TEAM_NAME'].map(lambda x:name_dict[x])
    df_nba = merge_one_df(df_nba)
    
    #去掉重复项并增加以天为单位的比赛时间
    df_lottery = df_lottery.drop_duplicates('betId')
    df_lottery['gameDate'] = df_lottery['matchTime'].map(lambda x:x.split(' ')[0])
    
    #按照比赛时间-球队的顺序排序
    df_nba.sort_values(['GAME_DATE','TEAM_NAME_中文'],inplace = True)
    df_lottery.sort_values(['gameDate','hostTeamName'],inplace = True)
    
    #pd.concat()是按照index进行的拼接，在sort_values()之后两个df的index被打乱了
    #直接拼接会出现类似下一段代码的情况
    df_nba.reset_index(drop=True,inplace=True)
    df_lottery.reset_index(drop=True,inplace=True)
    
    #拼接并保存（不保存index）
    df_result = pd.concat([df_nba,df_lottery],axis=1)
    df_result.to_excel(year.split('/')[-1]+'-records.xlsx',index=False)
    print(year)
    
df_result.head()

../数据收集/2005-06
../数据收集/2006-07
../数据收集/2007-08
../数据收集/2008-09
../数据收集/2009-10
../数据收集/2010-11
../数据收集/2011-12
../数据收集/2012-13
../数据收集/2013-14
../数据收集/2014-15
../数据收集/2015-16
../数据收集/2016-17


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,leagueId,leagueName,leagueType,letScoreTape,matchResult,matchStatus,matchTime,roundType,totalScoreTape,gameDate
0,22016.0,1.610613e+09,GSW,Golden State Warriors,21600003.0,2016-10-25,GSW vs. SAS,L,240.0,40.0,...,1,美职男篮,1,85,0,-50,2016-10-26 10:30,常规赛,2120,2016-10-26
1,22016.0,1.610613e+09,POR,Portland Trail Blazers,21600002.0,2016-10-25,POR vs. UTA,W,240.0,39.0,...,1,美职男篮,1,60,3,-50,2016-10-26 10:00,常规赛,1930,2016-10-26
2,22016.0,1.610613e+09,CLE,Cleveland Cavaliers,21600001.0,2016-10-25,CLE vs. NYK,W,240.0,45.0,...,1,美职男篮,1,95,3,-50,2016-10-26 07:30,常规赛,2045,2016-10-26
3,22016.0,1.610613e+09,PHI,Philadelphia 76ers,21600011.0,2016-10-26,PHI vs. OKC,L,240.0,36.0,...,1,美职男篮,1,-85,0,-50,2016-10-27 08:00,常规赛,2080,2016-10-27
4,22016.0,1.610613e+09,BOS,Boston Celtics,21600006.0,2016-10-26,BOS vs. BKN,W,240.0,48.0,...,1,美职男篮,1,130,3,-50,2016-10-27 07:30,常规赛,2115,2016-10-27


下面的例子体会一下merge、join、concat的区别：<br\>
- concat()以index为依据拼接
- join()、merge()以on指定的列为依据拼接

In [4]:
a=pd.DataFrame({'a':[1,2],'b':[3,4]},index=[1,2])
b=pd.DataFrame({'f':[1,2],'g':[44,55]})
# a=a.reset_index(drop=True)
pd.concat([a,b],axis=1)

,a,b,f,g
0,NaN,NaN,1.0,44.0
1,1.0,3.0,2.0,55.0
2,2.0,4.0,NaN,NaN


In [5]:
pd.concat([a,b])

,a,b,f,g
1,1.0,3.0,NaN,NaN
2,2.0,4.0,NaN,NaN
0,NaN,NaN,1.0,44.0
1,NaN,NaN,2.0,55.0


In [6]:
pd.merge(a,b,left_on='a',right_on='f')

,a,b,f,g
0,1,3,1,44
1,2,4,2,55


In [7]:
a.join(b,how='outer')

,a,b,f,g
0,NaN,NaN,1.0,44.0
1,1.0,3.0,2.0,55.0
2,2.0,4.0,NaN,NaN
